In [5]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")



import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

In [6]:
df = pd.read_csv(r'book_info.csv')
df.drop(['Title', 'Author', 'Average Rating', 'No of ratings', 'No of reviews', 'No of pages', 'No of awards', 'Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
print(df.shape)
df.isna().sum()

(7200, 2)


summary     893
Genre      1274
dtype: int64

In [7]:
initial = df.shape[0]

In [8]:
df.dropna(inplace=True)
df.shape

(5903, 2)

In [9]:
df['Genre'].unique()

array(['fantasy', 'young-adult', 'fiction', 'historical', 'non-fiction',
       'sequential-art', 'science-fiction', 'humor', 'horror', 'thriller',
       'romance', 'autobiography', 'history', 'mystery', 'drama',
       'suspense', 'womens-fiction', 'poetry', 'music', 'polyamorous',
       'christian', 'short-stories', 'food-and-drink', 'travel',
       'adventure', 'christian-fiction', 'biography', 'cultural',
       'business', 'paranormal', 'science-fiction-fantasy',
       'realistic-fiction', 'new-adult', 'art', 'dark', 'classics',
       'parenting', 'contemporary', 'childrens', 'philosophy',
       'psychology', 'science', 'erotica', 'spirituality', 'politics',
       'plays', 'lgbt', 'epic', 'religion', 'environment', 'lds',
       'health', 'cities', 'education', 'economics', 'self-help',
       'holiday', 'crime', 'novels', 'feminism', 'media-tie-in',
       'adult-fiction', 'neurodiversity', 'christianity', 'games',
       'academic', 'animals', 'westerns', 'world-war-ii',


As there are a large number of genre categories, we will club the similar ones together-
1. fantasy - shapeshifters, science-fiction-fantasy
2. young-adult - new-adult
3. fiction - womens-fiction, sequential-art, short-stories, childrens, christian-fiction, adult-fiction, realistic-fiction,
contemporary, drama, anthologies, comics
4. historical - history
5. romance
6. mystery - thriller, crime, war, suspense, adventure
7. science-fiction
8. classics
9. horror - paranormal
10. non-fiction - politics, biography, philosophy, self-help, autobiography, cultural, science, music, business, psychology, 
parenting, health, sports, travel, art, lgbt, holiday, food-and-drink, economics, animals, feminism, marriage, education,
humanities, academic, language, sexuality, literature, sports-and-games, amish, computer-science, games, environment, 
family-law, sociology, american, military-history, architecture, leadership, eastern-africa, inspirational, writing, cities, 
relationships, neurodiversity, world-war-ii, 
11. religion - spirituality, christian, christianity, lds


Some of the categories have very few books in them. Thus we'll be dropping all such categories-

reference                    
epic                         
folklore              
neurodiversity        
westerns                
dark                
plays                      
polyamorous          
european-literature         
erotica                  
humor                   
poetry                      
novels                      
media-tie-in               


In [10]:
df["Genre"].replace({"history":"historical", "shapeshifters":"fantasy", "science-fiction-fantasy":"fantasy", "new-adult":"young-adult", "thriller":"mystery", "crime":"mystery", "war":"mystery", "suspense":"mystery", "adventure":"mystery", "paranormal":"horror", "spirituality":"religion", "christian":"religion", "christianity":"religion", "lds":"religion"}, inplace=True)
for i in ["womens-fiction","sequential-art","short-stories","childrens","christian-fiction","adult-fiction","realistic-fiction","contemporary","drama","anthologies","comics"]:
    df["Genre"].replace({i:"fiction"}, inplace=True)
for j in ["politics", "biography", "philosophy", "self-help", "autobiography", "cultural", "science", "music", "business", "psychology", "parenting", "health", "sports", "travel", "art", "lgbt", "holiday", "food-and-drink", "economics", "animals", "feminism", "marriage", "education", "humanities", "academic", "language", "sexuality", "literature", "sports-and-games", "amish", "computer-science", "games", "environment", "family-law", "sociology", "american", "military-history", "architecture", "leadership", "eastern-africa", "inspirational", "writing", "cities", "relationships", "neurodiversity", "world-war-ii"]:
    df["Genre"].replace({j:"non-fiction"}, inplace=True)

In [11]:
rejected = ["reference", "epic", "folklore", "neurodiversity", "westerns", "dark", "plays", "polyamorous", "european-literature", "erotica", "humor", "poetry", "novels", "media-tie-in"]
for i in rejected :
    indexvalue = df[df['Genre'] == i].index
    df.drop(indexvalue , inplace=True)

In [12]:
df['Genre'].value_counts()

fiction            1538
fantasy             928
non-fiction         812
young-adult         614
historical          447
mystery             353
romance             339
horror              223
science-fiction     201
classics            188
religion            116
Name: Genre, dtype: int64

In [13]:
df.shape

(5759, 2)

In [14]:
final = df.shape[0]
len(df['Genre'].unique())

11

Thus, now we have 11 genres of books in our dataset

In [15]:
#Percentage of data retained after cleaning
(final/initial)*100

79.98611111111111

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5759 entries, 0 to 7199
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   summary  5759 non-null   object
 1   Genre    5759 non-null   object
dtypes: object(2)
memory usage: 135.0+ KB


In [17]:
df.head()
df.astype(str)

,summary,Genre
0,"It's no longer safe for Harry at Hogwarts, so ...",fantasy
1,"Could you survive on your own in the wild, wit...",young-adult
2,"The unforgettable, heartbreaking story of the ...",fiction
3,Librarian's note: An alternate cover edition c...,historical
4,The war against Voldemort is not going well; e...,fantasy
...,...,...
7195,Violetâs sure of three things:\nâ¢ Its seni...,young-adult
7196,Perfection comes at a price.\n\nAs soon as the...,young-adult
7197,Oneâ¦ Twoâ¦ Threeâ¦ Fourâ¦\n\nI Declare Wa...,fantasy
7198,"Six days ago, astronaut Mark Watney became one...",science-fiction


In [18]:
genre = df['Genre'].tolist()

In [20]:
len(genre)

5759

In [47]:
import re
def decontracted(phrase):
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [48]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [49]:
from bs4 import BeautifulSoup
from tqdm import tqdm
preprocessed_summary = []
for text in tqdm(df['summary'].values):
    text = re.sub(r"http\S+", "", str(text))
    text = BeautifulSoup(text, 'lxml').get_text()
    text = decontracted(text)
    text = re.sub("\S*\d\S*", "", text).strip()
    text = re.sub('[^A-Za-z]+', ' ', text)
    text = ' '.join(e.lower() for e in text.split() if e.lower() not in stopwords)
    preprocessed_summary.append(text.strip())

100%|████████████████████████████████████████████████████████████████████████████| 5759/5759 [00:05<00:00, 1120.65it/s]


In [53]:
len(preprocessed_summary)

5759

In [50]:
#Printing a random summary
preprocessed_summary[67]

'winter night dr david henry forced blizzard deliver twins son born first perfectly healthy yet daughter born sees immediately syndrome rationalizing need protect norah wife makes split second decision alter lives forever asks nurse take winter night dr david henry forced blizzard deliver twins son born first perfectly healthy yet daughter born sees immediately syndrome rationalizing need protect norah wife makes split second decision alter lives forever asks nurse take baby away institution never reveal secret caroline nurse cannot leave infant instead disappears another city raise child begins story unfolds quarter century two families ignorant yet bound fateful decision made long ago winter night norah henry knows daughter died birth remains inconsolable grief weighs heavily marriage paul son raises best house grown cold mourning meanwhile phoebe lost daughter grows sunny child vibrant young woman whose mother loves fiercely'

In [58]:
i=0
list_of_summaries=[]
for i in preprocessed_summary:
    list_of_summaries.append(i.split())

In [54]:
df['clean_summary'] = preprocessed_summary
df.isna().sum()

In [62]:
from gensim.models import Word2Vec
w2v_model=Word2Vec(list_of_summaries,min_count=3,vector_size=50, workers=4)

#Printing out vectors for some words
print(w2v_model.wv.most_similar('interesting'))
print('*'*50)
print(w2v_model.wv.most_similar('page'))

[('pants', 0.9916547536849976), ('buy', 0.9883843660354614), ('teach', 0.9883801341056824), ('catch', 0.9882172346115112), ('bethany', 0.9878074526786804), ('usually', 0.9875310659408569), ('simply', 0.9874577522277832), ('sleep', 0.9872955679893494), ('throw', 0.9869775176048279), ('racing', 0.9866356253623962)]
**************************************************
[('imaginative', 0.9857005476951599), ('presents', 0.982516884803772), ('beautifully', 0.9821913242340088), ('based', 0.9809738993644714), ('inspired', 0.980826199054718), ('groundbreaking', 0.9793399572372437), ('divergent', 0.9770607948303223), ('canterbury', 0.9758207201957703), ('definitive', 0.975761890411377), ('masterpiece', 0.9749545454978943)]


In [63]:
w2v_words = w2v_model.wv.index_to_key
vocab_len = len(w2v_model.wv)
print("total number of words in all summaries(that occured atleast thrice)",vocab_len)
print("some of these words are: ", w2v_model.wv.index_to_key[0:50])

total number of words in all summaries(that occured atleast thrice) 22630
some of these words are:  ['one', 'life', 'not', 'new', 'world', 'love', 'story', 'book', 'novel', 'family', 'first', 'time', 'no', 'years', 'two', 'old', 'man', 'year', 'young', 'people', 'never', 'even', 'author', 'like', 'must', 'lives', 'war', 'would', 'find', 'father', 'way', 'mother', 'back', 'ever', 'york', 'girl', 'woman', 'could', 'home', 'day', 'de', 'best', 'secret', 'us', 'times', 'american', 'past', 'long', 'human', 'three']


In [64]:
w2v_model.wv.vectors.shape

(22630, 50)

In [65]:
# S = ["abc def pqr", "def def def abc", "pqr pqr def"]
model = TfidfVectorizer()
model.fit(preprocessed_summary)
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(model.get_feature_names(), list(model.idf_)))

In [67]:
# TF-IDF weighted Word2Vec
tfidf_features = model.get_feature_names() # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

tfidf_sentence_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for summary in tqdm(list_of_summaries): # for each review/sentence 
    sum_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in summary: # for each word in w2v_worda review/sentence
        if word in w2v_words[0:len(w2v_words)] and word in tfidf_features:
            vec = w2v_model.wv[word]  
#             tf_idf = tf_idf_matrix[row, tfidf_feat.index(word)]
            # to reduce the computation we are 
            # dictionary[word] = idf value of word in whole courpus
            # sent.count(word) = tf valeus of word in this review
            tf_idf = dictionary[word]*(summary.count(word)/len(summary))
            tfidf_w2v = vec.dot(tf_idf)
            sum_vec += tfidf_w2v
            weight_sum += tf_idf
    if weight_sum != 0:
        final_summary_vector = sum_vec/weight_sum
    tfidf_sentence_vectors.append(final_summary_vector)
    row += 1

100%|██████████████████████████████████████████████████████████████████████████████| 5759/5759 [27:38<00:00,  3.47it/s]


In [69]:
len(tfidf_sentence_vectors)

5759

In [2]:
file_name = "summary_vectors.pkl"

In [70]:
file = open(file_name, "wb")
pickle.dump(tfidf_sentence_vectors, file)
file.close()

In [3]:
file = open(file_name, "rb")
vectors_list = pickle.load(file)
file.close()

In [4]:
len(vectors_list)

5759

In [73]:
df['summary_vector'] = vectors_list
df.isna().sum()

summary           0
Genre             0
clean_summary     0
summary_vector    0
dtype: int64

In [75]:
df.head()

,summary,Genre,clean_summary,summary_vector
0,"It's no longer safe for Harry at Hogwarts, so ...",fantasy,no longer safe harry hogwarts best friends ron...,"[-0.2675593665783395, -0.199968195994287, 0.29..."
1,"Could you survive on your own in the wild, wit...",young-adult,could survive wild every one make sure not liv...,"[-0.31312476778645093, -0.14755315433847316, 0..."
2,"The unforgettable, heartbreaking story of the ...",fiction,unforgettable heartbreaking story unlikely fri...,"[0.029256872705477034, 0.4390025424121535, 0.1..."
3,Librarian's note: An alternate cover edition c...,historical,librarian note alternate cover edition found n...,"[-0.19804477450831615, -0.016911188453721745, ..."
4,The war against Voldemort is not going well; e...,fantasy,war voldemort not going well even muggle gover...,"[-0.22790307719440325, -0.04706283251267159, 0..."


In [21]:
x = vectors_list
y = genre

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [24]:
x_train, x_test, y_train, y_test =train_test_split(x,y, test_size=0.2, random_state=0)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_train = np.array(x_train)
x_test = np.array(x_test)

In [25]:
opt_val = k_fold_cross_validation(x_train,y_train)

NameError: name 'k_fold_cross_validation' is not defined